In [1]:
#!pip install html5lib 

In [2]:
import requests

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)

In [5]:
data

<Response [429]>

## Scraping our first page with requests

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(data.text)

In [8]:
standings_table = soup.select('table.stats_table')[0]

IndexError: list index out of range

## Parsing our first page with requests

In [ ]:
links = standings_table.find_all('a') 

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if '/squads/' in l]

In [ ]:
links

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls = team_urls[0]

In [ ]:
data = requests.get(team_urls)

## Extract match stats using pandas and requests

In [ ]:
import pandas as pd
matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]

In [ ]:
matches.head()

## Get match shooting stts with requests and pandas

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
data

In [ ]:
shooting = pd.read_html(data.text, match = "Shooting")[0]

In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.columns

## Cleaning and merging scraped data with pandas

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")

In [ ]:
team_data.shape

In [ ]:
shooting.shape

In [ ]:
matches.shape

## Scraping data for multiple season and teams with forloops

In [ ]:
years = list(range(2022, 2019, -1))
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    if len(soup.select('table.stats_table')) == 0:
        continue
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        print(team_url)
        
        data = requests.get(team_url)
        try :
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        except ValueError:
            print(data.text)
            continue
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        data = requests.get(f"https://fbref.com{links[0]}")
        try:
            shooting = pd.read_html(data.text, match="Shooting")[0]
        except ValueError:
            print(links[0])
            print(data.text)
            continue
            
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("match_premier_league.csv")